In [7]:
import random
import math
import matplotlib.pyplot as plt
import numpy as np
from ipywidgets import interact, IntSlider, FloatSlider, Dropdown

# Função de Fitness
def fitness_function(y):
    """ Função para maximizar """
    return y + abs(math.sin(32 * y))

# Geração da população
def generate_population(POPULATION_SIZE):
    """ Gera uma população de tamanho POPULATION_SIZE com cromossomos representando números float """
    population = []
    for _ in range(POPULATION_SIZE):
        chromosome = random.uniform(0, math.pi)  # Gera números float aleatórios entre 0 e pi
        population.append(chromosome)
    return population

# Operadores de Crossover
def crossover_average(parent1, parent2):
    """ Crossover por média """
    weight = random.random()
    child1 = weight * parent1 + (1 - weight) * parent2
    child2 = (1 - weight) * parent1 + weight * parent2
    # Garantir que os filhos estejam dentro do intervalo [0, pi]
    child1 = max(0, min(child1, math.pi))
    child2 = max(0, min(child2, math.pi))
    return child1, child2

def crossover_single_point(parent1, parent2):
    """ Crossover de ponto único (não muito relevante para floats, mas pode ser simulado) """
    child1 = (parent1 + parent2) / 2
    child2 = (parent1 + parent2) / 2
    # Garantir que os filhos estejam dentro do intervalo [0, pi]
    child1 = max(0, min(child1, math.pi))
    child2 = max(0, min(child2, math.pi))
    return child1, child2

# Operadores de Mutação
def mutate_random(chromosome, mutation_rate):
    """ Mutação aleatória """
    if random.random() < mutation_rate:
        chromosome += random.uniform(-1.0, 1.0)
        # Garantir que o cromossomo permaneça dentro do intervalo [0, pi]
        chromosome = max(0, min(chromosome, math.pi))
    return chromosome

def mutate_perturb(chromosome, mutation_rate):
    """ Mutação por perturbação """
    if random.random() < mutation_rate:
        chromosome += random.gauss(0, 1)
        # Garantir que o cromossomo permaneça dentro do intervalo [0, pi]
        chromosome = max(0, min(chromosome, math.pi))
    return chromosome

# Operadores de Seleção
def select_roulette(population, fitness_values, selection_rate):
    """ Seleção por roleta """
    selected_population = []
    selection_count = int(selection_rate * len(population))
    if sum(fitness_values) > 0:  # Evitar divisão por zero
        fitness_values = [f / sum(fitness_values) for f in fitness_values]  # Normalizar fitness
    for _ in range(selection_count):
        idx = random.choices(range(len(population)), weights=fitness_values)[0]
        selected_population.append(population[idx])
    return selected_population

def select_tournament(population, fitness_values, selection_rate):
    """ Seleção por torneio """
    selected_population = []
    selection_count = int(selection_rate * len(population))
    for _ in range(selection_count):
        candidates = random.sample(list(zip(population, fitness_values)), k=3)
        winner = max(candidates, key=lambda x: x[1])[0]
        selected_population.append(winner)
    return selected_population

# Algoritmo Genético com Armazenamento para Visualização
def genetic_algorithm(population_size, generations, crossover_rate, mutation_rate, selection_rate,
                      crossover_method, mutation_method, selection_method):
    population = generate_population(population_size)
    best_fitness_values = []
    average_fitness_values = []  # Armazena a aptidão média de cada geração
    generation_data = []  # Armazena dados de cada geração

    for generation in range(generations):
        fitness_values = [fitness_function(chromosome) for chromosome in population]
        
        # Ajuste de fitness para valores positivos
        min_fitness = min(fitness_values)
        if min_fitness < 0:
            fitness_values = [f - min_fitness + 1 for f in fitness_values]
        
        # Calcular e armazenar a aptidão média
        average_fitness = np.mean(fitness_values)
        average_fitness_values.append(average_fitness)
        
        # Armazenar dados para a geração atual
        generation_data.append((population, fitness_values))
        
        best_fitness = max(fitness_values)
        best_fitness_values.append(best_fitness)
        # print(f'Generation {generation}: best fitness = {best_fitness}, average fitness = {average_fitness}')
        
        # Seleção
        population = selection_method(population, fitness_values, selection_rate)
        
        # Crossover e mutação
        next_generation = []
        while len(next_generation) < population_size:
            parent1, parent2 = random.sample(population, 2)
            if random.random() < crossover_rate:
                child1, child2 = crossover_method(parent1, parent2)
            else:
                child1, child2 = parent1, parent2
            next_generation.extend([mutation_method(child1, mutation_rate), mutation_method(child2, mutation_rate)])
        
        # Garantir que a próxima geração não exceda o tamanho da população
        population = next_generation[:population_size]
    
    return best_fitness_values, average_fitness_values, generation_data

# Função para plotar aptidão média ao longo das gerações
def plot_average_fitness(average_fitness_values):
    plt.figure(figsize=(10, 6))
    plt.plot(average_fitness_values, label='Aptidão Média')
    plt.title('Aptidão Média da População ao Longo das Gerações')
    plt.xlabel('Geração')
    plt.ylabel('Aptidão Média')
    plt.legend()
    plt.grid(True)
    plt.show()

def plot_fitness_values(best_fitness_values):
    plt.figure(figsize=(10, 6))
    plt.plot(best_fitness_values)
    plt.title('Evolução da Função de Fitness')
    plt.xlabel('Geração')
    plt.ylabel('Melhor Valor de Fitness')
    plt.grid(True)
    plt.show()

def plot_fitness_vs_chromosome(generation_data):
    num_generations = len(generation_data)
    plt.figure(figsize=(15, 8))
    
    for generation, (population, fitness_values) in enumerate(generation_data):
        plt.scatter(population, fitness_values, label=f'Geração {generation}', alpha=0.5)
    
    plt.title('Aptidão em Função do Cromossomo para Diversas Gerações')
    plt.xlabel('Cromossomo')
    plt.ylabel('Aptidão')
    plt.legend()
    plt.grid(True)
    plt.show()

# Função para rodar o algoritmo e plotar os gráficos
def run_ga(population_size, generations, crossover_rate, mutation_rate, selection_rate,
           crossover_method, mutation_method, selection_method):
    crossover_methods = {
        'Média': crossover_average,
        'Ponto Único': crossover_single_point
    }
    mutation_methods = {
        'Aleatória': mutate_random,
        'Perturbação': mutate_perturb
    }
    selection_methods = {
        'Roleta': select_roulette,
        'Torneio': select_tournament
    }
    
    best_fitness_values, average_fitness_values, generation_data = genetic_algorithm(
        population_size, generations, crossover_rate, mutation_rate, selection_rate,
        crossover_methods[crossover_method], mutation_methods[mutation_method], selection_methods[selection_method]
    )
    
    plot_average_fitness(average_fitness_values)
    plot_fitness_values(best_fitness_values)
    plot_fitness_vs_chromosome(generation_data)
    
# Widgets interativos
interact(run_ga,
         population_size=IntSlider(min=10, max=500, step=10, value=100, description='População:'),
         generations=IntSlider(min=1, max=600, step=1, value=10, description='Gerações:'),
         crossover_rate=FloatSlider(min=0.0, max=1.0, step=0.01, value=0.7, description='Crossover:'),
         mutation_rate=FloatSlider(min=0.0, max=1.0, step=0.01, value=0.05, description='Mutação:'),
         selection_rate=FloatSlider(min=0.0, max=1.0, step=0.01, value=0.5, description='Seleção:'),
         crossover_method=Dropdown(options=['Média', 'Ponto Único'], value='Média', description='Crossover:'),
         mutation_method=Dropdown(options=['Aleatória', 'Perturbação'], value='Aleatória', description='Mutação:'),
         selection_method=Dropdown(options=['Roleta', 'Torneio'], value='Roleta', description='Seleção:')
)


interactive(children=(IntSlider(value=100, description='População:', max=500, min=10, step=10), IntSlider(valu…

<function __main__.run_ga(population_size, generations, crossover_rate, mutation_rate, selection_rate, crossover_method, mutation_method, selection_method)>